In [3]:
import pandas as pd
import numpy as np
import re

##### All the data below was extracted from the Interactive Data Application of the Bureau of Economic Analysis of the United States:  https://www.bea.gov/itable/regional-gdp-and-personal-income

### Personal income per capita by Metropolitan Statistical Area (MSA)

### We extract the data from our S3 bucket

In [1]:
from private.s3_aws import access_key, secret_access_key

In [4]:
df = pd.read_csv(f"s3://rawdatagrupo07/personal_income_2011-2021bymetro.csv",
    storage_options={
        "key": access_key,
        "secret": secret_access_key
    },
)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GeoFips  385 non-null    int64 
 1   GeoName  385 non-null    object
 2   2011     385 non-null    int64 
 3   2012     385 non-null    int64 
 4   2013     385 non-null    int64 
 5   2014     385 non-null    int64 
 6   2015     385 non-null    int64 
 7   2016     385 non-null    int64 
 8   2017     385 non-null    int64 
 9   2018     385 non-null    int64 
 10  2019     385 non-null    int64 
 11  2020     385 non-null    int64 
 12  2021     385 non-null    int64 
dtypes: int64(12), object(1)
memory usage: 39.2+ KB


In [4]:
df.head()

,GeoFips,GeoName,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,998,United States (Metropolitan Portion),44206,46096,46251,48459,50426,51424,53453,55796,58347,61866,66392
1,10180,"Abilene, TX (Metropolitan Statistical Area)",36880,38410,39467,41974,40497,39784,40812,43279,45676,49210,55556
2,10420,"Akron, OH (Metropolitan Statistical Area)",40126,41380,42215,44055,45343,45991,47732,49631,51827,55232,58735
3,10500,"Albany, GA (Metropolitan Statistical Area)",32487,32700,32515,33225,34145,34633,35521,36293,38118,42006,45619
4,10540,"Albany-Lebanon, OR (Metropolitan Statistical A...",32374,33489,34006,36251,38379,39372,40980,43389,44501,48357,52695


In [5]:
# we remove the first row as it is an average of the rows below and reset the index
df.drop(index=0, inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,GeoFips,GeoName,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,10180,"Abilene, TX (Metropolitan Statistical Area)",36880,38410,39467,41974,40497,39784,40812,43279,45676,49210,55556
1,10420,"Akron, OH (Metropolitan Statistical Area)",40126,41380,42215,44055,45343,45991,47732,49631,51827,55232,58735
2,10500,"Albany, GA (Metropolitan Statistical Area)",32487,32700,32515,33225,34145,34633,35521,36293,38118,42006,45619
3,10540,"Albany-Lebanon, OR (Metropolitan Statistical A...",32374,33489,34006,36251,38379,39372,40980,43389,44501,48357,52695
4,10580,"Albany-Schenectady-Troy, NY (Metropolitan Stat...",45774,47143,47847,49407,51809,52558,55039,56501,59477,63951,67788


In [6]:
# we split the GeoName column by the comma, returning the Metropolitan Area and the State/s in different columns
geo = df['GeoName'].str.split(pat=',', expand=True)
geo
msa = geo[0]
state = geo[1].str.split(expand=True)
state = state[0]


In [7]:
# we add the msa and state columns to the dataframe and drop the geoname column
df = pd.concat([msa, state, df], axis=1)
df.drop(['GeoName'], axis=1, inplace=True )
df.columns = ['MSA', 'State', 'Fips_Code', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

In [9]:
# we turn the years headers into values
metro = pd.melt(df, id_vars=['MSA','State','Fips_Code'],value_vars=['2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021'],var_name='Year',value_name='Personal_Income')

In [10]:
metro

,MSA,State,Fips_Code,Year,Personal_Income
0,Abilene,TX,10180,2011,36880
1,Akron,OH,10420,2011,40126
2,Albany,GA,10500,2011,32487
3,Albany-Lebanon,OR,10540,2011,32374
4,Albany-Schenectady-Troy,NY,10580,2011,45774
...,...,...,...,...,...
4219,Yakima,WA,49420,2021,50872
4220,York-Hanover,PA,49620,2021,57870
4221,Youngstown-Warren-Boardman,OH-PA,49660,2021,48904
4222,Yuba City,CA,49700,2021,51174


### Personal income per capita by County

In [5]:
#df = pd.read_csv(f"s3://rawdatagrupo07/personal_income_2011-2016bycounty.csv",
#    storage_options={
#        "key": access_key,
#        "secret": secret_access_key
#    },
#)

In [6]:
df = pd.read_csv(f"s3://rawdatagrupo07/personal_income_2017-2021bycounty.csv",
    storage_options={
        "key": access_key,
        "secret": secret_access_key
    },
)

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GeoFips  3140 non-null   int64 
 1   GeoName  3140 non-null   object
 2   2017     3140 non-null   object
 3   2018     3140 non-null   object
 4   2019     3140 non-null   object
 5   2020     3140 non-null   object
 6   2021     3140 non-null   object
dtypes: int64(1), object(6)
memory usage: 171.8+ KB


In [151]:
df.head()

,GeoFips,GeoName,2017,2018,2019,2020,2021
0,1001,"Autauga, AL",40185,41243,42846,45248,48347
1,1003,"Baldwin, AL",44870,46247,48380,51348,54659
2,1005,"Barbour, AL",32167,33328,34870,37120,40428
3,1007,"Bibb, AL",28912,30060,31800,34598,36892
4,1009,"Blount, AL",33922,35291,36542,38351,42634


In [152]:
# we extract the state from de geoname colum to create a new one
df['State'] = df['GeoName'].str.split(pat=',')
df['State'] = df['State'].apply(lambda x: x[-1])
df


,GeoFips,GeoName,2017,2018,2019,2020,2021,State
0,1001,"Autauga, AL",40185,41243,42846,45248,48347,AL
1,1003,"Baldwin, AL",44870,46247,48380,51348,54659,AL
2,1005,"Barbour, AL",32167,33328,34870,37120,40428,AL
3,1007,"Bibb, AL",28912,30060,31800,34598,36892,AL
4,1009,"Blount, AL",33922,35291,36542,38351,42634,AL
...,...,...,...,...,...,...,...,...
3135,56037,"Sweetwater, WY",51396,55377,56619,55188,56934,WY
3136,56039,"Teton, WY",228049,221209,278682,290674,318297,WY
3137,56041,"Uinta, WY",38363,39833,41297,42704,44157,WY
3138,56043,"Washakie, WY",45908,49292,51388,57228,60235,WY


In [153]:
# we extract de county from the geoname column and drop the geoname column
df['County'] = df['GeoName'].str.split(pat=',')
df['County'] = df['County'].apply(lambda x: x[0])
df['County'] = df['County'].str.split(pat='+')
df['County'] = df['County'].apply(lambda x: x[0])
#df = df[['County', 'State', 'GeoFips', '2011', '2012', '2013', '2014', '2015', '2016']]
df = df[['County', 'State', 'GeoFips', '2017', '2018', '2019', '2020', '2021']]

In [154]:
# we turn the years headers into values and rename a geofips column

#county = pd.melt(df, id_vars=['County','State','GeoFips'],value_vars=['2011', '2012', '2013', '2014', '2015', '2016'],var_name='Year',value_name='Personal_Income')
county = pd.melt(df, id_vars=['County','State','GeoFips'],value_vars=['2017', '2018', '2019', '2020', '2021'],var_name='Year',value_name='Personal_Income')
county.rename(columns={'GeoFips': 'Fips_Code'}, inplace=True)

In [155]:
# we normalise some values
county['County'] = county['County'].astype(str).apply(lambda x: re.sub('\(.+\)','',x))
county['County'] = county['County'].str.rstrip()
county['State'] = county['State'].str.replace('*', '', regex=False)
county['State'] = county['State'].str.lstrip()
county['Personal_Income'] = county['Personal_Income'].replace('(NA)', None)

In [156]:
# we delete rows with null income 
county.isnull().sum()
county.dropna(inplace=True)
county['Personal_Income'] = county['Personal_Income'].astype(int)
county.reset_index(inplace=True, drop=True)

### Personal income per capita by State

In [7]:
df = pd.read_csv(f"s3://rawdatagrupo07/personal_income_2011-2021bystate.csv",
    storage_options={
        "key": access_key,
        "secret": secret_access_key
    },
)

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GeoFips  60 non-null     int64 
 1   GeoName  60 non-null     object
 2   2011     60 non-null     int64 
 3   2012     60 non-null     int64 
 4   2013     60 non-null     int64 
 5   2014     60 non-null     int64 
 6   2015     60 non-null     int64 
 7   2016     60 non-null     int64 
 8   2017     60 non-null     int64 
 9   2018     60 non-null     int64 
 10  2019     60 non-null     int64 
 11  2020     60 non-null     int64 
 12  2021     60 non-null     int64 
dtypes: int64(12), object(1)
memory usage: 6.2+ KB


In [161]:
df.head()

,GeoFips,GeoName,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,0,United States,42747,44548,44798,46887,48725,49613,51550,53786,56250,59765,64143
1,1000,Alabama,35010,35824,36014,37055,38531,39014,40223,41539,43288,46179,49769
2,2000,Alaska *,52569,53708,52812,55867,57575,56278,57189,59618,61316,62756,65813
3,4000,Arizona,35321,36545,37139,38756,40334,41473,43497,45466,48124,52327,55487
4,5000,Arkansas,34279,36582,36677,38749,39968,40873,41890,43384,44324,47154,50625


In [164]:
# we remove the first row as it is an average of the rows below and reset the index
df.drop(index=0, inplace=True)
df.reset_index(inplace=True, drop=True)

In [166]:
# we normalise some values
df['GeoName'] = df['GeoName'].str.replace('*', '', regex=False)
df['GeoName'] = df['GeoName'].str.lstrip()
df.head()

,GeoFips,GeoName,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,1000,Alabama,35010,35824,36014,37055,38531,39014,40223,41539,43288,46179,49769
1,2000,Alaska,52569,53708,52812,55867,57575,56278,57189,59618,61316,62756,65813
2,4000,Arizona,35321,36545,37139,38756,40334,41473,43497,45466,48124,52327,55487
3,5000,Arkansas,34279,36582,36677,38749,39968,40873,41890,43384,44324,47154,50625
4,6000,California,45557,48121,48502,51266,54546,56560,58804,61508,64919,70647,76614


In [168]:
# we turn the years headers into values and rename a geofips column
state = pd.melt(df, id_vars=['GeoName','GeoFips'],value_vars=['2011', '2012', '2013', '2014', '2015', '2016','2017', '2018', '2019', '2020', '2021'],var_name='Year',value_name='Personal_Income')
state.rename(columns={'GeoFips': 'Fips_Code', 'GeoName': 'State'}, inplace=True)
state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State            649 non-null    object
 1   Fips_Code        649 non-null    int64 
 2   Year             649 non-null    object
 3   Personal_Income  649 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 20.4+ KB
